# Previsão de inadimplência de empréstimo com recursos simplificados


# Análise_de_Risco_Crédito

**Arquivo credit_risk.csv** Importado do Kaggle

Descrições das colunas :

ID : identificador exclusivo para cada solicitante de empréstimo.

Idade : Idade do solicitante do empréstimo.

Renda : Renda do solicitante do empréstimo.

Casa : Situação de propriedade de casa própria (Própria, Hipoteca, Aluguel).

Emp_Length : Tempo de emprego em anos.

Intenção : Finalidade do empréstimo (por exemplo, educação, reforma da casa).

Valor : valor do empréstimo solicitado.

Taxa : taxa de juros do empréstimo.

Status : status de aprovação do empréstimo (totalmente pago, cobrado, atual).

Percent_Income : Valor do empréstimo como porcentagem da receita.

Inadimplência : se o solicitante já deixou de pagar um empréstimo anteriormente (Sim, Não). = DEFAULT

Cred_Length : Comprimento do histórico de crédito do solicitante.

# Importando Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, r2_score
from sklearn.model_selection import train_test_split
import math
from sklearn.neighbors import DistanceMetric, KNeighborsClassifier
from sklearn import neighbors
from sklearn.metrics import mean_squared_error,mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.cluster import KMeans
from sklearn import tree
import graphviz
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Importando arquivos do Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Input file
input_file = '/content/drive/My Drive/vincit/credit_risk.csv'
df = pd.read_csv(input_file, sep = ",",low_memory=False)
#df = pd.read_csv('../../Datasets/credit_risk.csv', sep = ",", low_memory=False)

# Carregando os dados e fazendo pré-processamento

In [ ]:

df

,Id,Age,Income,Home,Emp_length,Intent,Amount,Rate,Status,Percent_income,Default,Cred_length
0,0,22,59000,RENT,123.0,PERSONAL,35000,16.02,1,0.59,Y,3
1,1,21,9600,OWN,5.0,EDUCATION,1000,11.14,0,0.10,N,2
2,2,25,9600,MORTGAGE,1.0,MEDICAL,5500,12.87,1,0.57,N,3
3,3,23,65500,RENT,4.0,MEDICAL,35000,15.23,1,0.53,N,2
4,4,24,54400,RENT,8.0,MEDICAL,35000,14.27,1,0.55,Y,4
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,32576,57,53000,MORTGAGE,1.0,PERSONAL,5800,13.16,0,0.11,N,30
32577,32577,54,120000,MORTGAGE,4.0,PERSONAL,17625,7.49,0,0.15,N,19
32578,32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,35000,10.99,1,0.46,N,28
32579,32579,56,150000,MORTGAGE,5.0,PERSONAL,15000,11.48,0,0.10,N,26


# Analisando as colunas do Dataset e fazendo as transformações necessárias

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              32581 non-null  int64  
 1   Age             32581 non-null  int64  
 2   Income          32581 non-null  int64  
 3   Home            32581 non-null  object 
 4   Emp_length      31686 non-null  float64
 5   Intent          32581 non-null  object 
 6   Amount          32581 non-null  int64  
 7   Rate            29465 non-null  float64
 8   Status          32581 non-null  int64  
 9   Percent_income  32581 non-null  float64
 10  Default         32581 non-null  object 
 11  Cred_length     32581 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 3.0+ MB


In [ ]:
df.describe()

,Id,Age,Income,Emp_length,Amount,Rate,Status,Percent_income,Cred_length
count,32581.000000,32581.000000,3.258100e+04,31686.000000,32581.000000,29465.000000,32581.000000,32581.000000,32581.000000
mean,16290.006139,27.734600,6.607485e+04,4.789686,9589.371106,11.011695,0.218164,0.170203,5.804211
std,9405.479594,6.348078,6.198312e+04,4.142630,6322.086646,3.240459,0.413006,0.106782,4.055001
min,0.000000,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,8145.000000,23.000000,3.850000e+04,2.000000,5000.000000,7.900000,0.000000,0.090000,3.000000
50%,16290.000000,26.000000,5.500000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000
75%,24435.000000,30.000000,7.920000e+04,7.000000,12200.000000,13.470000,0.000000,0.230000,8.000000
max,32780.000000,144.000000,6.000000e+06,123.000000,35000.000000,23.220000,1.000000,0.830000,30.000000


Verificamos que a tabela tem 32.581 linhas, verificamos também que não necessitamos da coluna ID


Retirando a coluna IDS


In [ ]:
dropar=["Id"]

In [ ]:
df = df.drop(columns=dropar, axis=1)

In [ ]:
df

,Age,Income,Home,Emp_length,Intent,Amount,Rate,Status,Percent_income,Default,Cred_length
0,22,59000,RENT,123.0,PERSONAL,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,35000,14.27,1,0.55,Y,4
...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,5800,13.16,0,0.11,N,30
32577,54,120000,MORTGAGE,4.0,PERSONAL,17625,7.49,0,0.15,N,19
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,35000,10.99,1,0.46,N,28
32579,56,150000,MORTGAGE,5.0,PERSONAL,15000,11.48,0,0.10,N,26


#  Verificando e eliminando valores duplicados

In [ ]:
df.duplicated().sum()

165

In [ ]:
df.drop_duplicates(keep="first",inplace=True)
df.duplicated().sum()

0

# Verificando e eliminando valores nulos

In [ ]:
df.isna().sum().sum()

3982

Aproximadamente 10% eram valores nulos, então poderia ser feita uma análise exploratória para verificar se esses dados poderiam ter impacto no resultado então a opção foi, simplesmente retirá-los.


In [ ]:
df.dropna(inplace=True)
df.isna().sum().sum()

0

Vericamos que há variáveis do tipo objeto então iremos usar Hot Encoder para transforma-las em numéricas

In [ ]:
# label encoder para transformar todas as colunas em numéricas
le = preprocessing.LabelEncoder()
for column in df.columns:
    if(df[column].dtypes=='object'):
        df[column] = le.fit_transform(df[column])

df.head()

,Age,Income,Home,Emp_length,Intent,Amount,Rate,Status,Percent_income,Default,Cred_length
0,22,59000,3,123.0,4,35000,16.02,1,0.59,1,3
1,21,9600,2,5.0,1,1000,11.14,0,0.10,0,2
2,25,9600,0,1.0,3,5500,12.87,1,0.57,0,3
3,23,65500,3,4.0,3,35000,15.23,1,0.53,0,2
4,24,54400,3,8.0,3,35000,14.27,1,0.55,1,4


In [ ]:
# Matriz para estabelecer a Correlação
corr = df.corr()
corr.style.background_gradient(cmap = 'coolwarm')

,Age,Income,Home,Emp_length,Intent,Amount,Rate,Status,Percent_income,Default,Cred_length
Age,1.000000,0.178987,-0.035156,0.165624,0.036478,0.054246,0.010170,-0.024091,-0.040782,0.007181,0.859621
Income,0.178987,1.000000,-0.201591,0.136427,0.002469,0.264942,-0.001346,-0.140456,-0.251487,-0.002653,0.116622
Home,-0.035156,-0.201591,1.000000,-0.232544,0.010356,-0.130304,0.136704,0.208148,0.140541,0.062408,-0.025153
Emp_length,0.165624,0.136427,-0.232544,1.000000,0.020593,0.110934,-0.056607,-0.082852,-0.055033,-0.028179,0.146486
Intent,0.036478,0.002469,0.010356,0.020593,1.000000,-0.003424,-0.002559,-0.064754,0.000308,-0.002493,0.036259
Amount,0.054246,0.264942,-0.130304,0.110934,-0.003424,1.000000,0.146026,0.114153,0.577708,0.040714,0.045334
Rate,0.010170,-0.001346,0.136704,-0.056607,-0.002559,0.146026,1.000000,0.339995,0.123441,0.500174,0.014562
Status,-0.024091,-0.140456,0.208148,-0.082852,-0.064754,0.114153,0.339995,1.000000,0.380000,0.182397,-0.016559
Percent_income,-0.040782,-0.251487,0.140541,-0.055033,0.000308,0.577708,0.123441,0.380000,1.000000,0.036503,-0.029690
Default,0.007181,-0.002653,0.062408,-0.028179,-0.002493,0.040714,0.500174,0.182397,0.036503,1.000000,0.007988


### Separando o alvo (Status) dos atributos, criando variável X (dados que usaremos para prever y=alvo)

In [ ]:
target = pd.DataFrame(df, columns=["Status"])

In [ ]:
X = df.drop(df.columns[7], axis=1)
y = target

In [ ]:
X

,Age,Income,Home,Emp_length,Intent,Amount,Rate,Percent_income,Default,Cred_length
0,22,59000,3,123.0,4,35000,16.02,0.59,1,3
1,21,9600,2,5.0,1,1000,11.14,0.10,0,2
2,25,9600,0,1.0,3,5500,12.87,0.57,0,3
3,23,65500,3,4.0,3,35000,15.23,0.53,0,2
4,24,54400,3,8.0,3,35000,14.27,0.55,1,4
...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,0,1.0,4,5800,13.16,0.11,0,30
32577,54,120000,0,4.0,4,17625,7.49,0.15,0,19
32578,65,76000,3,3.0,2,35000,10.99,0.46,0,28
32579,56,150000,0,5.0,4,15000,11.48,0.10,0,26


In [ ]:
y

,Status
0,1
1,0
2,1
3,1
4,1
...,...
32576,0
32577,0
32578,1
32579,0


# Dividir em conjuntos de treinamento e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) # Dividindo o dataset 80% para treino e 20% para teste

### Regressão Linear

In [ ]:
from sklearn.linear_model import LinearRegression

#Criando objeto de regressão
lr = LinearRegression()

#Treinando regressão
lr.fit(X_train,y_train)

LinearRegression()

In [ ]:
value_pred = lr.predict(X_test)
value_pred # Valores previstos para os dados de teste

array([[ 0.65702094],
       [-0.06588539],
       [ 0.3128917 ],
       ...,
       [ 0.16753894],
       [-0.16349494],
       [ 0.15349363]])

In [ ]:
def results_regression(y_test,y_pred):
    mse = mean_squared_error(y_test,y_pred)
    print(f"mse: {mse}")
    rmse = math.sqrt(mse)
    print(f"rmse: {rmse}")
    mae = mean_absolute_error(y_test,y_pred)
    print(f"mae: {mae}")
    mape = mean_absolute_percentage_error(y_test,y_pred)
    print(f"mape: {mape}")
    r2 = r2_score(y_test,y_pred)
    print(f"r2_score {r2}")


In [ ]:
results_regression(y_test, value_pred)

mse: 0.12084144459730185
rmse: 0.34762256054131735
mae: 0.2687426982096021
mape: 664086057864769.2
r2_score 0.28114312170270894


# Random Forest

In [ ]:
# Inicializando Classificador Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=20)

In [ ]:
# Treinando o modelo
rf_classifier.fit(X_train, y_train)

<ipython-input-39-57333542a21f>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=20)

In [ ]:
# Fazendo predições com os dados de teste
y_pred = rf_classifier.predict(X_test)

In [ ]:
# Relatório de classificação e acurácia
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print("Acurácia:", accuracy)
print("Relatório de Classificação:\n", classification_rep)

Acurácia: 0.9219435186809332
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.92      0.99      0.95      4482
           1       0.93      0.69      0.79      1219

    accuracy                           0.92      5701
   macro avg       0.92      0.84      0.87      5701
weighted avg       0.92      0.92      0.92      5701

